# LangChain (and LangGraph)


<img src="img/langchain_book.jpeg" width="200px"/>

### Some references

* https://learning.oreilly.com/library/view/learning-langchain/9781098167271/preface01.html

* https://learning.oreilly.com/library/view/learning-langchain/9781098167271/ch01.html

* https://github.com/vinodvpillai/langchain-conversation-memory-examples

* https://www.ibm.com/think/tutorials/prompt-chaining-langchain

* https://datascientistsdiary.com/langchain-vs-langgraph/
----

## Setup

In [1]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain, SequentialChain
from langchain_core.messages import SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate
)
from langchain_openai import ChatOpenAI


from dotenv import load_dotenv

_ = load_dotenv()

## Example 1: Adding memory to simple chat completion

In [2]:


prompt = ChatPromptTemplate(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", 
                                  return_messages=True)

legacy_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=prompt,
    memory=memory,
)

result1 = legacy_chain.invoke({"text": "My name is Timon"})

print(result1)

print('\n----\n')

result2 = legacy_chain.invoke({"text": "Here are some other names: Amina, Akira, Carlos, Fatima, Ji-hoon, Leila, Maria, Nia, Priya, Raphael, Sanaa, Sofia, Tao, Victor, Yara"})

print(result2)

print('\n----\n')

result3 = legacy_chain.invoke({"text": "What was my name?"})

print(result3)

/tmp/ipykernel_306552/2773693126.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history",
/tmp/ipykernel_306552/2773693126.py:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  legacy_chain = LLMChain(


{'text': 'Nice to meet you, Timon! How can I assist you today?', 'chat_history': [HumanMessage(content='My name is Timon', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you, Timon! How can I assist you today?', additional_kwargs={}, response_metadata={})]}

----

{'text': 'Those are some lovely names! If you would like, I can provide some information or assistance related to these names. Just let me know how I can help!', 'chat_history': [HumanMessage(content='My name is Timon', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you, Timon! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Here are some other names: Amina, Akira, Carlos, Fatima, Ji-hoon, Leila, Maria, Nia, Priya, Raphael, Sanaa, Sofia, Tao, Victor, Yara', additional_kwargs={}, response_metadata={}), AIMessage(content='Those are some lovely names! If you would like, I can provide some information or assistance related

## Example 2: A multi-stage chain



In [3]:


model = ChatOpenAI(model="gpt-4o-mini")


prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [4]:
question_generator.invoke("happy")

ChatPromptValue(messages=[HumanMessage(content='What is the color of Banana and the flag of Colombia?', additional_kwargs={}, response_metadata={})])

## Example 3 - Analyzing review text

* Adapted from: https://www.ibm.com/think/tutorials/prompt-chaining-langchain

In [5]:
keyword_prompt = PromptTemplate(
    input_variables = ["text"],
    template="Extract the most important keywords from the following text:\n{text}\n\nKeywords:"
)

sentiment_prompt = PromptTemplate(
    input_variables = ["keywords"],
    template="Using the following keywords, summarize the sentiment:\nKeywords: {keywords}\n\nSentiment Summary:"
)


refine_prompt = PromptTemplate(
    input_variables = ["sentiment_summary"],
    template="Refine the following sentiment summary to make it concise and precise:\n{sentiment_summary}\n\nRefined summary"
)

In [6]:
VERBOSE = True

In [7]:
keyword_chain = LLMChain(
    llm = model,
    prompt = keyword_prompt,
    output_key = "keywords",
    verbose = VERBOSE
)

sentiment_chain = LLMChain(
    llm = model,
    prompt = sentiment_prompt,
    output_key = "sentiment_summary",
    verbose = VERBOSE
)

refine_chain = LLMChain(
    llm = model,
    prompt = refine_prompt,
    output_key = "refined_summary",
    verbose = VERBOSE
)

In [8]:
workflow = SequentialChain(
        chains = [keyword_chain, sentiment_chain, refine_chain],
        input_variables=["text"],
        output_variables=["refined_summary"],
        verbose=VERBOSE,
        return_all=True
)

In [9]:

review1 = '''
Somewhere in the past year, ChatGPT has gone from "cool, interesting, amusing" to a massively valuable work assistant capable of writing Python scripts, analysing data, and doing lots, lots, more. The key to this? The rapid evolution of OpenAI's GPT models and making my first forays into prompt engineering.

If I thought I was getting good, though, this book took me reminded me that I'm just scratching the surface. Halfway through the first chapter I was already furiously scribbling notes in the margins for what I could do better with my prompt writing and by the end of the text I felt like I had gotten a very good grounding - not just in GPTs specifically but in the bigger picture of how these hugely powerful tools were trained and came to maturity.

I imagine that few will argue with my assertion that there is lots of hyperbole and "noise" in the AI space right now which, as ever, makes it hard to pick out the signal from the noise. Which is precisely why I sought out an O'Reilly title and I'm very glad that I did.

Thorough, excellent, and I hope that this edition will be the first of many. As this rapidly maturing field scales and matures I think that prompt engineering will be an essential discipline to master. Pick up this text to get a good foothold on things.
'''

In [10]:
review2 = '''
I applaud the authors for putting forth a comprehensive introduction in a rapidly evolving space. I absorbed a lot, helpful as I was developing a prototype — using open source methods.

And that’s where I was disappointed. The LLM and examples are highly adapted to OpenAI’s GPT-x and the bulky LangChain framework, something not obvious until you dig in to the book. Sure, this may be where newbie demand was when the authors began writing. But as the open source models and OpenAI alternatives gain speed (e.g. Llama 3.1, Groq, etc.) this book may quickly need an updated and expanded version to stay relevant.
'''

In [11]:
result = workflow.run({"text": review2})
print(result)

/tmp/ipykernel_306552/985787591.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = workflow.run({"text": review2})




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Extract the most important keywords from the following text:

I applaud the authors for putting forth a comprehensive introduction in a rapidly evolving space. I absorbed a lot, helpful as I was developing a prototype — using open source methods.

And that’s where I was disappointed. The LLM and examples are highly adapted to OpenAI’s GPT-x and the bulky LangChain framework, something not obvious until you dig in to the book. Sure, this may be where newbie demand was when the authors began writing. But as the open source models and OpenAI alternatives gain speed (e.g. Llama 3.1, Groq, etc.) this book may quickly need an updated and expanded version to stay relevant.


Keywords:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Using the following keywords, summarize the sentiment:
Keywords: - Authors
- Comprehensive introduction
- Evolving space
- Prototype

In [12]:
result

'The sentiment reflects a positive outlook on advancements in natural language processing and AI, highlighting large language models (LLMs) like OpenAI’s GPT-x, Llama 3.1, and Groq. The focus on open-source methods and the LangChain framework suggests a collaborative environment catering to newcomers. Terms like "updated version" and "expanded version" indicate a commitment to continuous improvement and innovation, emphasizing the need for individuals and organizations to stay relevant in this rapidly evolving field. Overall, the sentiment conveys enthusiasm for progress and accessibility in AI technologies.'

In [13]:
workflow.run({"text": review2})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Extract the most important keywords from the following text:

I applaud the authors for putting forth a comprehensive introduction in a rapidly evolving space. I absorbed a lot, helpful as I was developing a prototype — using open source methods.

And that’s where I was disappointed. The LLM and examples are highly adapted to OpenAI’s GPT-x and the bulky LangChain framework, something not obvious until you dig in to the book. Sure, this may be where newbie demand was when the authors began writing. But as the open source models and OpenAI alternatives gain speed (e.g. Llama 3.1, Groq, etc.) this book may quickly need an updated and expanded version to stay relevant.


Keywords:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Using the following keywords, summarize the sentiment:
Keywords: 1. Authors
2. Comprehensive introduction
3. Rapidly evolving space


'The sentiment surrounding Large Language Models (LLMs) and open-source methods is notably positive and optimistic. There is a growing interest among newcomers for accessible resources, indicating a demand for comprehensive introductions to this rapidly evolving field. Developments like updated versions of GPT-x, Llama 3.1, and alternatives to OpenAI demonstrate the dynamic nature of technology, with a wider range of options available through frameworks like LangChain and tools from organizations like Groq. Overall, the sentiment reflects excitement about innovation, accessibility, and collaboration within the community, underscoring the importance of ongoing projects and tools.'